In [3]:
!pip install torch transformers
import torch
import numpy as np
from scipy.special import softmax

In [4]:
from google.colab import drive
import pandas as pd
from IPython.display import display
from google.colab import drive
drive.mount('/content/drive')
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/data/random_selected_summary.csv'
df = pd.read_csv(csv_file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model.eval()


def score_article(article):
    # Tokenize the input article
    inputs = tokenizer(article, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the logits and apply softmax to get probabilities
    scores = outputs.logits[0].numpy()
    scores = softmax(scores)

    # Get the sentiment label
    sentiment = np.argmax(scores)

    # Define sentiment labels
    labels = ['Negative', 'Neutral', 'Positive']

    return labels[sentiment], scores

# Initialize a list to store results
results = []

# Loop through articles and score them
for _, article in df.head(10).iterrows():
    # Score the cleaned article
    article_predicted_class, article_class_probabilities = score_article(article['cleaned_article'])

    # Score the summary
    summary_predicted_class, summary_class_probabilities = score_article(article['gpt4_summary'])

    # Append results to the list
    results.append({
        "Article Title": article['title'],  # Assuming you want to use the article's title
        "Article Predicted Class": article_predicted_class,
        "Article Class Probabilities": article_class_probabilities,
        "GPT Predicted Class": summary_predicted_class,
        "GPT Class Probabilities": summary_class_probabilities
    })

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Display the results in tabular format
display(df)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly id

,Article Title,Article Predicted Class,Article Class Probabilities,GPT Predicted Class,GPT Class Probabilities
0,Poland abortion: Polish women look for help in...,Neutral,"[0.23114845, 0.7413644, 0.027487205]",Negative,"[0.5752412, 0.40550116, 0.019257676]"
1,Hong Kong Cardinal advocates for promoting a c...,Neutral,"[0.008529517, 0.6685069, 0.32296357]",Neutral,"[0.019603483, 0.6642657, 0.3161309]"
2,Sex Education in the Philippines - The Borgen ...,Neutral,"[0.028562682, 0.8699873, 0.10145]",Neutral,"[0.03552598, 0.7980512, 0.16642278]"
3,"When a fetal scan showed problems, she fled Id...",Neutral,"[0.21159431, 0.7582101, 0.03019549]",Negative,"[0.7993527, 0.19286892, 0.0077783675]"
4,Prioritizing the Neglected Areas of SRHR in Ca...,Neutral,"[0.04012358, 0.8849569, 0.074919514]",Neutral,"[0.06709984, 0.7739449, 0.15895523]"
5,Abortion policy activism heats up for Roe v. W...,Neutral,"[0.025680033, 0.88952494, 0.084794976]",Neutral,"[0.016985903, 0.72043985, 0.26257423]"
6,Barbara Kruger’s 'Your Body is a Battleground'...,Neutral,"[0.38287562, 0.5929085, 0.024215886]",Neutral,"[0.18584721, 0.76148653, 0.05266627]"
7,Abortion 'practically banned in Turkey',Negative,"[0.61025226, 0.37998286, 0.009764943]",Negative,"[0.7803826, 0.2130225, 0.006594942]"
8,"Mitch McConnell, Senate Republicans Vote to St...",Neutral,"[0.029951394, 0.84299266, 0.12705593]",Negative,"[0.59414667, 0.39589497, 0.009958371]"
9,DeSantis signs 15-week abortion ban into law d...,Neutral,"[0.03308042, 0.9154518, 0.051467776]",Neutral,"[0.15613751, 0.8052616, 0.038600873]"
